In [1]:
from pathlib import Path

organization = "org-NaDNa5c5In5LspIF45qqArRr" # This is mine. You should change it!

keyfile = Path.home() / '.openai' / 'api.key'
with open(keyfile) as f:
    api_key = f.readline().strip()

In [2]:
import openai
openai.organization = organization
openai.api_key = api_key

In [3]:
gpt_model_name = "gpt-3.5-turbo"

In [4]:
def basic_completion(user_prompt, max_tokens=50):
    x = openai.ChatCompletion.create(
        model = gpt_model_name,
        messages = [
            {'role': 'user', 'content': user_prompt}
        ],
        max_tokens = max_tokens
    )
    return x.choices[0].message

In [5]:
# basic_completion("Write a haiku about being a dungeon master")

In [6]:
system_prompt = '''
You are a dungeon master assistant. You will create and improvise world content, behaviors, and events.

World lore and game mechanics should be compatible with dungeons and dragons standard fifth edition (5e).

All response MUST be in valid YAML format, and MUST NOT contain any other characters.
The user expects to be able to parse each raw response as YAML. The response should NOT be formatted as Markdown.
'''

import yaml

def system_completion(user_prompt):
    amended_user_prompt = f"""
    {user_prompt}

    The response MUST be in valid YAML format, and MUST NOT contain any other characters.
    The user expects to be able to parse each raw response as YAML. The response should NOT be formatted as Markdown.
    """

    call = openai.ChatCompletion.create(
        model = gpt_model_name,
        messages = [
            {'role': 'system', 'content': system_prompt},
            {'role': 'user', 'content': amended_user_prompt}
        ],
    )
    raw_message = call.choices[0].message['content']
    parsed_message = yaml.safe_load(raw_message)
    return (raw_message, parsed_message)


In [7]:
#(raw_name, parsed_name) = system_completion("Generate a name for a fantasy world")
#print(raw_name)

In [17]:
def range(pair): return f"at least {pair[0]} and no greater than {pair[1]}"

def geography_prompt(continents=(3,3), regions=(4,7)): return f'''
Generate the geography of a fictional fantasy world, that will be the setting for the game.

There MUST be {range(continents)} continents in the world.
There MUST be {range(regions)} regions per continent.

The response must be in the following structure. 

continents:
    name: <string>
    location: <string>
    regions:
        name: <string>
        location: <string>
        topography: <string>
        climate: <string>
        biome: <string>
        size: <string>
'''

geography_prompt()

'\nGenerate the geography of a fictional fantasy world, that will be the setting for the game.\n\nThere MUST be at least 3 and no greater than 3 continents in the world.\nThere MUST be at least 4 and no greater than 7 regions per continent.\n\nThe response must be in the following structure. \n\ncontinents:\n    name: <string>\n    location: <string>\n    regions:\n        name: <string>\n        location: <string>\n        topography: <string>\n        climate: <string>\n        biome: <string>\n        size: <string>\n'

In [18]:
(raw_geography, parsed_geography) = system_completion(geography_prompt())
print(raw_geography)

continents:
  - name: "Auroria"
    location: "Northern Hemisphere"
    regions:
      - name: "Frostlands"
        location: "Northwest Auroria"
        topography: "Frozen tundras and icy mountains"
        climate: "Arctic"
        biome: "Taiga"
        size: "Large"
      - name: "Highvale"
        location: "Northeast Auroria"
        topography: "High-altitude mountains and deep valleys"
        climate: "Subarctic"
        biome: "Alpine"
        size: "Medium"
      - name: "Golden Plains"
        location: "Southeast Auroria"
        topography: "Rolling hills and vast grasslands"
        climate: "Temperate"
        biome: "Grasslands"
        size: "Large"
      - name: "Darkwood"
        location: "Southwest Auroria"
        topography: "Thick forests and swamps"
        climate: "Humid subtropical"
        biome: "Forest"
        size: "Medium"

  - name: "Eremus"
    location: "Southern Hemisphere"
    regions:
      - name: "Duneswept"
        location: "Southwest Eremu

In [19]:
geography_continent_context = []

for continent in parsed_geography['continents']:
    name = continent['name']
    location_in_world = continent['location']
    def get_biome(region): return region['biome']
    biomes = map(get_biome, continent['regions'])
    biomes = ', '.join(biomes)
    line = f"The continent {name} located in the {location_in_world} with {biomes} biomes."
    geography_continent_context.append(line)

geography_continent_context = '\n'.join(geography_continent_context)
print(geography_continent_context)

The continent Auroria located in the Northern Hemisphere with Taiga, Alpine, Grasslands, Forest biomes.
The continent Eremus located in the Southern Hemisphere with Desert, Forest, Volcanic, Savanna biomes.
The continent Mareterra located in the Equator with Island, Jungle, Mountain, Desert biomes.


In [25]:
def civilizations_prompt(geography_context, civilizations=(4,5), cities=(3,6)): return f"""
Generate the major fictional fantasy civilizations in the world and their major cities.

There MUST be {range(civilizations)} civilizations in the world.
There MUST be {range(cities)} cities per civilization.

The civlizations and settlements should exist in the world's geopgrahy, defined by the below section:

```
{geography_context}
```

The response must be in the following structure.

civilizations:
    location: <reference, name of continent in world geopraphy>
    cities:
        location: <reference, name of region in world geography>
        city_economy_description: <string>
        name: <string>
    ruling_faction: <string>
    name: <string>
"""

civilizations_prompt(raw_geography)

'\nGenerate the major fictional fantasy civilizations in the world and their major cities.\n\nThere MUST be at least 4 and no greater than 5 civilizations in the world.\nThere MUST be at least 3 and no greater than 6 cities per civilization.\n\nThe civlizations and settlements should exist in the world\'s geopgrahy, defined by the below section:\n\n```\ncontinents:\n  - name: "Auroria"\n    location: "Northern Hemisphere"\n    regions:\n      - name: "Frostlands"\n        location: "Northwest Auroria"\n        topography: "Frozen tundras and icy mountains"\n        climate: "Arctic"\n        biome: "Taiga"\n        size: "Large"\n      - name: "Highvale"\n        location: "Northeast Auroria"\n        topography: "High-altitude mountains and deep valleys"\n        climate: "Subarctic"\n        biome: "Alpine"\n        size: "Medium"\n      - name: "Golden Plains"\n        location: "Southeast Auroria"\n        topography: "Rolling hills and vast grasslands"\n        climate: "Temperate

In [26]:
(raw_civilizations, parsed_civlizations) = system_completion(civilizations_prompt(raw_geography))
print(raw_civilizations)

civilizations:
  - location: "Auroria"
    cities:
      - location: "Frostlands"
        city_economy_description: "Mining and blacksmithing."
        name: "Frosthold"
      - location: "Frostlands"
        city_economy_description: "Fishing and trade."
        name: "Icyport"
      - location: "Highvale"
        city_economy_description: "Agriculture and animal husbandry."
        name: "Highmount"
      - location: "Highvale"
        city_economy_description: "Tourism and cultural exchange."
        name: "Skyhaven"
      - location: "Golden Plains"
        city_economy_description: "Commerce and craftsmanship."
        name: "Gildedbridge"
      - location: "Golden Plains"
        city_economy_description: "Religious pilgrimage and scholarly exchange."
        name: "Brightgleam"
      - location: "Darkwood"
        city_economy_description: "Timber and leatherworking."
        name: "Darkhaven"
      - location: "Darkwood"
        city_economy_description: "Alchemy and arcane res

In [42]:
# validation

lines = []

for civilization in parsed_civlizations['civilizations']:
    name = civilization['name']
    lines.append(f"Civilization: {name}")
    lines.append(f"\tRuled by: {civilization['ruling_faction']}")
    location_continent = civilization['location']
    matches = [c for c in parsed_geography['continents'] if c['name'] == location_continent]
    if not len(matches) == 1:
        raise Exception(f"for civ {name} couldn't find continent {location_continent}")
    continent = matches[0]
    lines.append(f"\tLocated on the continent {location_continent} at {continent['location']}")
    for city in civilization['cities']:
        location_region = city['location']
        matches = [r for r in continent['regions'] if r['name'] == location_region]
        if not len(matches) == 1:
            raise Exception(f"for city {city['name']} couldn't find region {location_region} in {location_continent}")
        region = matches[0]
        lines.append(f"\t\tCity: {city['name']} (Region: {region['name']}, {region['location']}, {region['topography']}, {region['biome']})")
        lines.append(f"\t\t\t{city['city_economy_description']}")

map_context = '\n'.join(lines)
print(map_context)

Civilization: The Alliance of Four Realms
	Ruled by: The Council of Four
	Located on the continent Auroria at Northern Hemisphere
		City: Frosthold (Region: Frostlands, Northwest Auroria, Frozen tundras and icy mountains, Taiga)
			Mining and blacksmithing.
		City: Icyport (Region: Frostlands, Northwest Auroria, Frozen tundras and icy mountains, Taiga)
			Fishing and trade.
		City: Highmount (Region: Highvale, Northeast Auroria, High-altitude mountains and deep valleys, Alpine)
			Agriculture and animal husbandry.
		City: Skyhaven (Region: Highvale, Northeast Auroria, High-altitude mountains and deep valleys, Alpine)
			Tourism and cultural exchange.
		City: Gildedbridge (Region: Golden Plains, Southeast Auroria, Rolling hills and vast grasslands, Grasslands)
			Commerce and craftsmanship.
		City: Brightgleam (Region: Golden Plains, Southeast Auroria, Rolling hills and vast grasslands, Grasslands)
			Religious pilgrimage and scholarly exchange.
		City: Darkhaven (Region: Darkwood, Sout

Hah, the map looks like shit. Don't both with the openai Image api (yet)